## Step 1: Loading a Random Customer Dataset

In [0]:
# Load dataset into a DataFrame
customer_df = spark.read.csv('/FileStore/tables/customer_dataset.csv', header=True, inferSchema=True)
customer_df.show()

+----------+-------------+-------------+--------------------+
|CustomerID|         Name|      Address|               Email|
+----------+-------------+-------------+--------------------+
|         1|     John Doe|   123 Elm St|    john@example.com|
|         2|   Jane Smith|   456 Oak St|    jane@example.com|
|         3|  Alice Brown|  789 Pine St|   alice@example.com|
|         4|Charlie Black| 234 Maple St| charlie@example.com|
|         5|    Eve White| 567 Birch St|     eve@example.com|
|         6|     John Doe|   123 Elm St|   john2@example.com|
|         7|   Jane Smith|   456 Oak St|   jane2@example.com|
|         8|  Alice Brown|  789 Pine St|  alice2@example.com|
|         9|Charlie Black| 234 Maple St|charlie2@example.com|
|        10|    Eve White| 567 Birch St|    eve2@example.com|
|        11|Michael Green| 890 Cedar St| michael@example.com|
|        12|   Sarah Blue|678 Spruce St|   sarah@example.com|
|        13|Michael Green| 890 Cedar St|michael2@example.com|
|       

## Step 2: Identify Duplicate Customers Based on Name and Address

In [0]:
from pyspark.sql.functions import col, count

# group by name and address and count occurences
duplicates_df = customer_df.groupBy("Name", "Address").count().filter(col("count") > 1)
duplicates_df.show()

+-------------+--------------+-----+
|         Name|       Address|count|
+-------------+--------------+-----+
|Michael Green|  890 Cedar St|    5|
|    Eve White|  567 Birch St|    3|
|   Sarah Blue| 678 Spruce St|    2|
|Charlie Black|  234 Maple St|    3|
| Laura Purple| 345 Poplar St|    4|
|    David Red|901 Hickory St|    2|
|     John Doe|    123 Elm St|    3|
|  Alice Brown|   789 Pine St|    3|
|  Anna Yellow| 345 Poplar St|    2|
|   Jane Smith|    456 Oak St|    3|
+-------------+--------------+-----+



## Assign Master ID to Matching Customers

Entire code has been written below for clarity and can run as a independent cell


In [0]:
from pyspark.sql.functions import col, concat_ws, row_number, monotonically_increasing_id
from pyspark.sql import Window

# Initialize Spark session (if not already initialized), some website mention its automatically created when we open a notebook
spark = SparkSession.builder \
    .appName("CustomerDeduplication") \
    .getOrCreate()

# Load dataset into a DataFrame
customer_df = spark.read.csv('/FileStore/tables/customer_dataset.csv', header=True, inferSchema=True)

# Replace null values if any in Name and Address columns with empty strings
customer_df = customer_df.fillna({'Name': '', 'Address': ''})

# Create a CompositeKey column by concatenating Name and Address
customer_df_modified = customer_df.withColumn("CompositeKey", concat_ws(' ', col("Name"), col("Address")))
print("Added Composite key :")
customer_df_modified.show()

# Use dense_rank to assign the same MasterID to duplicates
windowSpec = Window.partitionBy("CompositeKey").orderBy(monotonically_increasing_id())
master_id_df = customer_df_modified.withColumn("MasterID", row_number().over(windowSpec))


print("Below are 2 variations of outputs with Master ID columns : ")
# Select relevant columns
result_df = master_id_df.select("CustomerID", "Name", "Address", "MasterID")
result_df.show()

# Merge the MasterID back into the original dataset
merged_df = customer_df_modified.join(result_df.select("CustomerID", "MasterID"), on="CustomerID", how="left")
merged_df.show()


Added Composite key :
+----------+-------------+-------------+--------------------+--------------------+
|CustomerID|         Name|      Address|               Email|        CompositeKey|
+----------+-------------+-------------+--------------------+--------------------+
|         1|     John Doe|   123 Elm St|    john@example.com| John Doe 123 Elm St|
|         2|   Jane Smith|   456 Oak St|    jane@example.com|Jane Smith 456 Oa...|
|         3|  Alice Brown|  789 Pine St|   alice@example.com|Alice Brown 789 P...|
|         4|Charlie Black| 234 Maple St| charlie@example.com|Charlie Black 234...|
|         5|    Eve White| 567 Birch St|     eve@example.com|Eve White 567 Bir...|
|         6|     John Doe|   123 Elm St|   john2@example.com| John Doe 123 Elm St|
|         7|   Jane Smith|   456 Oak St|   jane2@example.com|Jane Smith 456 Oa...|
|         8|  Alice Brown|  789 Pine St|  alice2@example.com|Alice Brown 789 P...|
|         9|Charlie Black| 234 Maple St|charlie2@example.com|Char